In [ ]:
import pandas as pd
import numpy as np

In [ ]:
bakery_data = pd.read_csv('../data/raw_data/french_bakery_sales.csv')
display(bakery_data)

In [ ]:
def format_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Creates a Copy of the Original DataFrame. Formats the DataFrame column names to lowercase and formats the 'article' column values to lowercase
    with underscores instead of spaces.

    Args:
        df (pd.DataFrame): The input DataFrame.

    Returns:
        pd.DataFrame: The modified DataFrame with lowercase column names and formatted 'Item' values.
    """
    
    # Create a copy of the original DataFrame
    formatted_df = df.copy()

    # Rename columns to lowercase
    formatted_df.columns = formatted_df.columns.str.lower()

    # Format 'article' column values to lowercase with underscores
    formatted_df['article'] = formatted_df['article'].str.lower().str.replace(' ', '_')

    return formatted_df

In [ ]:
format_dataframe(bakery_data)

In [ ]:
bakery_data = format_dataframe(bakery_data)
bakery_data.drop('unnamed: 0', axis=1, inplace=True)
bakery_data

In [ ]:
bakery_data['article'].unique()

In [ ]:
def drop_values_from_column(df: pd.DataFrame, column: str, values: list) -> pd.DataFrame:
    """
    Drops specified values from a given column in the DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column (str): The name of the column from which values need to be dropped.
        values (list): A list of values to be dropped from the column.

    Returns:
        pd.DataFrame: The modified DataFrame with dropped values from the specified column.
    """
    # Create a copy of the original DataFrame
    modified_df = df.copy()

    # Drop rows containing specified values from the column
    modified_df = modified_df[modified_df[column].isin(values) == False].reset_index(drop=True)

    return modified_df

In [ ]:
values_to_drop = ['coupe', 'traiteur', 'the', 'plat_6.50e', 'plat_7.60e', 'plat_7.00',
       'plat', 'plat_8.30e', 'formule_pate', 'pt_plateau_sale', 'reduction_sucrees_12', '.', 'platprepare6,50', 'platprepare5,50', 'platprepare7,00',
       'formule_plat_prepare', 'platprepare6,00', 'article_295', 'reduction_sucrees_24', 'gd_plateau_sale' ]

In [ ]:
modified_bakery_data = drop_values_from_column(bakery_data, 'article', values_to_drop)

display(modified_bakery_data)

In [ ]:
bakery_data = modified_bakery_data

In [ ]:
bakery_data.info()

In [ ]:
#drop rows with string 'divers' included in any column

bakery_data = bakery_data[~bakery_data.article.str.contains("divers")]

In [ ]:
#checking if the rows were dropped

bakery_data['article'].unique()

In [ ]:
bakery_data.info()

In [ ]:
def move_column_to_beginning(dataframe, column_name):
    """
    Moves a given column to the beginning of a DataFrame.

    Parameters:
        - dataframe (pandas.DataFrame): The DataFrame to modify.
        - column_name (str): The name of the column to move.

    Returns:
        pandas.DataFrame: The modified DataFrame with the specified column moved to the beginning.
    """
    # Identify the column name and store it in a variable
    column_to_move = dataframe[column_name]
    
    # Drop the column
    dataframe = dataframe.drop(column_name, axis=1)
    
    # Insert the column in the beggining as index 0 and axis 1
    dataframe.insert(0, column_name, column_to_move)
    
    return dataframe

In [ ]:
bakery_data = move_column_to_beginning(bakery_data, 'ticket_number')

In [ ]:
bakery_data.info()
bakery_data.describe(include=object).T

In [ ]:
def convert_column_to_float(df: pd.DataFrame, column_name: str, symbol_to_remove: str) -> pd.DataFrame:
    """
    Convert a column in a DataFrame from object to float64 type,
    remove a specified symbol, and substitute ',' with '.' in its values.
    
    Args:
        df (pd.DataFrame): The DataFrame containing the column.
        column_name (str): The name of the column to convert.
        symbol_to_remove (str): The symbol to remove from the column values.
    
    Returns:
        pd.DataFrame: The modified DataFrame with the specified column converted to float64 type,
                      the symbol removed from its values, and ',' substituted with '.'.
    """
    # Create a copy of the original DataFrame
    modified_df = df.copy()
    
    # Remove specified symbol from the column values in the copied DataFrame
    modified_df[column_name] = modified_df[column_name].str.replace(symbol_to_remove, '')
    
    # Substitute ',' with '.' in the column values in the copied DataFrame
    modified_df[column_name] = modified_df[column_name].str.replace(',', '.')
    
    # Convert the copied column to float64 type in the copied DataFrame
    modified_df[column_name] = modified_df[column_name].astype('float64')
    
    return modified_df


In [ ]:
converted_bakery_data = convert_column_to_float(bakery_data, 'unit_price', '€')

In [ ]:
display(converted_bakery_data)

In [ ]:
bakery_data = converted_bakery_data

In [ ]:
bakery_data.info()
round(bakery_data.describe(),2)

In [ ]:
bakery_data.describe(include=object)

In [ ]:
bakery_data = bakery_data.drop(bakery_data.index[bakery_data['quantity'] < 0])

In [ ]:
round(bakery_data.describe(),2)

In [ ]:
bakery_data.describe(include=object)

In [ ]:
bakery_data['unit_price'].unique()

In [ ]:
ordered_by_price_data = bakery_data.copy()

In [ ]:
display(ordered_by_price_data.sort_values('unit_price'))

In [ ]:
df_filtered = ordered_by_price_data[ordered_by_price_data['unit_price'] >= 0.01]

In [ ]:
display(df_filtered.sort_values('unit_price').head())

In [ ]:
bakery_data = df_filtered

In [ ]:
bakery_data.info()
round(bakery_data.describe(),2).T

In [ ]:
bakery_data.describe(include=object)

In [ ]:
bakery_data_table = bakery_data.copy()

In [ ]:
bakery_price_table = bakery_data_table[['article','unit_price']]
bakery_price_table = bakery_price_table.drop_duplicates()

In [ ]:
bakery_price_table = bakery_price_table.reset_index(drop=True)

In [ ]:
bakery_price_table = bakery_price_table.sort_values(by=['article'], ignore_index=True)
bakery_price_table = bakery_price_table.drop_duplicates(subset='article', keep="last").reset_index(drop=True)

In [ ]:
display(bakery_price_table)

In [ ]:
bakery_price_table.describe().T

In [ ]:
merged_df = bakery_data.merge(bakery_price_table, on='article', how='left')
merged_df['unit_price_x'].fillna(merged_df['unit_price_y'], inplace=True)
merged_df.drop(['unit_price_x'], axis=1, inplace=True)
merged_df.rename(columns={'unit_price_y': 'unit_price'}, inplace=True)

display(merged_df)

In [ ]:
bakery_data = merged_df
bakery_data['article_total'] = bakery_data['unit_price'] * bakery_data['quantity'] # create a new column for the product of unit_price and quantity

In [ ]:
display(bakery_data)

In [ ]:
tickets_total = bakery_data.groupby(by='ticket_number')['article_total'].sum()
tickets_total.columns = ['ticket_number', 'total_ticket']

In [ ]:
display(tickets_total)

In [ ]:
tickets_total.info()

In [ ]:
tickets_total = tickets_total.to_frame(name="total_ticket")

In [ ]:
display(tickets_total)

In [ ]:
tickets_total.info()
round(tickets_total.describe().T,2)

In [ ]:
bakery_data.to_csv('../data/clean_data/bakery_data.csv')
tickets_total.to_csv('../data/clean_data/tickets_total.csv')
bakery_price_table.to_csv('../data/clean_data/bakery_price_table.csv')